<a href="https://colab.research.google.com/github/sh-Gabriel/Atrial-Enlargement-Detection/blob/main/TCCJhonsons.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#imports

from google.colab import drive

import matplotlib.pyplot as plt
# from google.colab.patches import cv2_imshow

from PIL import Image
import numpy  as np
import pandas as pd
import cv2    as cv

import os
import glob
import csv
from itertools import islice
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

import tensorflow as tf

In [2]:
#mount and defines

drive.mount('/content/drive',force_remount=True)
PATH            = 'drive/MyDrive/TCC/Dataset_V1/'
norm            = "N"
doente          = "AAE_EP"
left            = "Torax_LAT_E"
right           = "Torax_LAT_D"
stop_n = stop_s = 6

# !git clone https://github.com/pjreddie/darknet.git
# !cd drive/MyDrive/TCC/darknet; make
!nvidia-smi --query-gpu=memory.total --format=csv
!nvidia-smi

Mounted at /content/drive
memory.total [MiB]
15109 MiB
Tue Oct  4 22:24:38 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                 

Falta descobrir como saporr de baixo funciona.

In [ ]:
#Monta uma rede VGG16
tf.keras.applications.vgg16.VGG16(
    include_top=True,
    weights='imagenet',
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000,
    classifier_activation='softmax'
)

In [ ]:
"testes"
stop_l = openImgs(doente, left)
stop_r = openImgs(doente, right)

os.listdir('drive/MyDrive/TCC/Dataset_V1/AAE_EP')

Coloquei os defs e a chamada do join_open_files tudo junto, porque basicamente eles fazem uma coisa só que é o join_open_files e assim, não precisa ficar rodando os defs de novo toda vez que muda alguma coisa

In [ ]:
from google.colab.patches import cv2_imshow

#1 chamada = 1 load em 1 classe com 1 lateralidade
# retorno do vetor com os nomes das imagens
def openImgs(classe,lateralidade):
  r_v = [] 
  for subdir, dirs, files in os.walk(PATH+classe):
    if (files and lateralidade in subdir):
      r_v += [subdir+'/'+files[0]]
  return shuffle(r_v, random_state = 42)

#Retorna vetores para entrada de modelo
#  X valores e Y labels
#  Y label da imagem (1 = presença de doença)

def join(lateralidade):
  #print("Accessing files...")
  X=[]
  Y=[]
  for f in islice(openImgs(norm, lateralidade), 0, stop_n):
      X += [f]
      Y += [0]

  for f in islice(openImgs(doente, lateralidade), 0, stop_s):
      X += [f]
      Y += [1]
  return X,Y

def join_open_files(lateralidade=left, reshape=True):
  X, y = join(lateralidade)
  Xl = []
  for i in X:
    # cv2_imshow(cv.imread(i, 0))
    if reshape:
      Xl.append(cv.resize(cv.imread(i, 0), (2928, 2328)))
    else:
      Xl.append(cv.imread(i,0))
  return Xl, y

X, y = join_open_files() #Coloca as imagens já abertas nos vetores de 'treino e teste'

In [ ]:
cv2_imshow(X[0])
# cv2_imshow(cv.resize(X[0], (2500, 2500)))

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (6,) + inhomogeneous part.
(arrumei o erro dando um resize e mudando a escala de cor na hora de abrir a imagen no join_open_files, depois disso dá um outro erro no fit dizendo que a dimensão tinha que ser 2 e é 3. Pra arrumar o erro de dimensão, forcei um shape e rashape no X.)

In [ ]:
print("Size da lista X " + str(len(X)))
print("Size do X pro split " + str(np.array(X).shape))
print("Size do reshape: ", str(np.array(X).reshape((nsamples,nx*ny)).shape));

# for x in X:
#   print(array(x[0]));

# cv2_imshow(X[5])
print(y_test)

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.5, random_state = 0)
# clf = svm.SVC(kernel='linear', C=1).fit(X_train, y_train)
# clf.score(X_test, y_test)
from sklearn import svm
#Leave one patient out using SVC from above
import numpy as np
from sklearn.model_selection import LeaveOneOut
def LOO(clf, X, y):
  loo = LeaveOneOut()
  loo.get_n_splits(X)
  nsamples, nx, ny = np.array(X).shape
  score = 0
  # print(X[:4])
  for train_index, test_index in loo.split(X):
    X_train, X_test = np.array(X).reshape((nsamples,nx*ny))[train_index], np.array(X).reshape((nsamples,nx*ny))[test_index]
    y_train, y_test = np.array(y)[train_index], np.array(y)[test_index]
    clf.fit(X_train, y_train)
    score += clf.score(X_test, y_test)

  print(score/12)
  return X_train, X_test, y_train, y_test
  
  
X_train, X_test, y_train, y_test = LOO(svm.SVC(kernel='linear', C= 1), X, y)

Verificar pra fazer algo customizado e alterar o classiicador por meio de um laço, fica bala

Antes tava fazendo 50% treino e 50% teste, por isso o 0.333333...

> Mount e Defines

>  Funções que abrem a imagem

> *   openImgs()
*   join()
*   joi_open_file()

> Teste utilizando uma Support Vector Classifier(SVC) linear


In [3]:
from sklearn.base import BaseEstimator, TransformerMixin
import random
from sklearn.pipeline import Pipeline
from google.colab.patches import cv2_imshow
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import decode_predictions
from sklearn import metrics
from tensorflow import keras

class CustomStandardScalerForCnn(TransformerMixin):
    def __init__(self, with_mean=True, with_std=True):
        self.with_mean = with_mean
        self.with_std = with_std
        self.mean_ = None
        self.std_ = None
        
    def fit(self, X, y=None):
        if self.with_mean:
            self.mean_ = X.mean()
        else:
            self.mean_ = 0
            
        if self.with_std:
            self.std_ = X.std()
        else:
            self.std_ = 1
        
        return self
    
    def transform(self, X):
        if self.mean_ and self.std_:
            return (X - self.mean_) / self.std_
        else:
            raise("CustomStandardScalerForCnn is not fitted")
            
    def inverse_transform(self, X):
        if self.with_std:
            X *= self.std_
        if self.with_mean:
            X += self.mean_
        return X

In [4]:
def VGG_inspired_build():
    clf = keras.models.Sequential([
        keras.layers.ZeroPadding2D((1,1), input_shape=(32, 32, 3)),
        keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
        keras.layers.ZeroPadding2D((1,1)),
        keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
        keras.layers.MaxPooling2D(pool_size=(2, 2)), # stride=2
        keras.layers.ZeroPadding2D((1,1)),
        keras.layers.Conv2D(filters=128, kernel_size=(3, 3), activation='relu'),
        keras.layers.ZeroPadding2D((1,1)),
        keras.layers.Conv2D(filters=128, kernel_size=(3, 3), activation='relu'),
        keras.layers.MaxPooling2D(pool_size=(2, 2)), # stride=2
        keras.layers.ZeroPadding2D((1,1)),
        keras.layers.Conv2D(filters=256, kernel_size=(3, 3), activation='relu'),
        keras.layers.ZeroPadding2D((1,1)),
        keras.layers.Conv2D(filters=256, kernel_size=(3, 3), activation='relu'),
        keras.layers.MaxPooling2D(pool_size=(2, 2)), # stride=2
        keras.layers.Flatten(),
        keras.layers.Dense(2048, activation='relu'),
        keras.layers.Dropout(0.75),
        keras.layers.Dense(2048, activation='relu'),
        keras.layers.Dropout(0.75),
        keras.layers.Dense(33, activation='softmax')
    ])
    
    
    clf.compile(optimizer=keras.optimizers.Adam(),
                loss='categorical_crossentropy',
                metrics=['accuracy'])

    return clf

In [33]:
from numpy import array;

gebriel = keras.applications.VGG16(input_shape=(224, 224, 3), include_top=False, weights="imagenet")

for layer in gebriel.layers: 
  print(layer.name)
  layer.trainable = False

last_layer = gebriel.get_layer('block5_pool')
last_output = last_layer.output
xd = tf.keras.layers.GlobalMaxPooling2D()(last_output)
xd = tf.keras.layers.Dense(512, activation='relu')(xd)
xd = tf.keras.layers.Dropout(0.5)(xd)
xd = tf.keras.layers.Dense(2, activation='sigmoid')(xd)


gemodelo = keras.Model(gebriel.input, xd)

gemodelo.compile(optimizer='adam', loss=tf.keras.losses.sparse_categorical_crossentropy, metrics=['acc'])



X, y = join_open_files() #Coloca as imagens já abertas nos vetores de 'treino e teste'

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

score = 0
gemodelo.compile(optimizer='adam',loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
print(X_train.shape, y_train.shape)
print(array(y_train))

# gemodelo.fit(X_train, y_train)
vgg_classifier = gemodelo.fit(X_train,y_train, epochs = 10,verbose = 1)

y_pred = gemodelo.evaluate(X_test)
print(y_test)
print(y_pred)

# import math
# print(math.ceil(y_pred))
# print(metrics.classification_report(y_test, y_pred))
#return X_train, X_test, y_train, y_test






input_29
block1_conv1
block1_conv2
block1_pool
block2_conv1
block2_conv2
block2_pool
block3_conv1
block3_conv2
block3_conv3
block3_pool
block4_conv1
block4_conv2
block4_conv3
block4_pool
block5_conv1
block5_conv2
block5_conv3
block5_pool
(8, 224, 224, 3) (8,)
[0 1 0 1 1 0 0 0]
Epoch 1/10


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


1/1 [==============================] - 1s 673ms/step - loss: 10.6830 - accuracy: 0.6250
Epoch 2/10
1/1 [==============================] - 0s 86ms/step - loss: 1.4901e-08 - accuracy: 1.0000
Epoch 3/10
1/1 [==============================] - 0s 79ms/step - loss: 9.1217 - accuracy: 0.8750
Epoch 4/10
1/1 [==============================] - 0s 55ms/step - loss: 1.5457 - accuracy: 0.8750
Epoch 5/10
1/1 [==============================] - 0s 54ms/step - loss: 17.0013 - accuracy: 0.7500
Epoch 6/10
1/1 [==============================] - 0s 52ms/step - loss: 8.2134 - accuracy: 0.5000
Epoch 7/10
1/1 [==============================] - 0s 51ms/step - loss: 6.0628 - accuracy: 0.7500
Epoch 8/10
1/1 [==============================] - 0s 55ms/step - loss: 0.6069 - accuracy: 0.8750
Epoch 9/10
1/1 [==============================] - 0s 52ms/step - loss: 1.6078 - accuracy: 0.8750
Epoch 10/10
1/1 [==============================] - 0s 138ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00
[1 1 0 1]
[0.0, 0.0]


In [16]:
from sklearn.model_selection import LeaveOneOut

#1 chamada = 1 load em 1 classe com 1 lateralidade
# retorno do vetor com os nomes das imagens
def openImgs(classe,lateralidade):
  r_v = [] 
  for subdir, dirs, files in os.walk(PATH+classe):
    if (files and lateralidade in subdir):
      r_v += [subdir+'/'+files[0]]
  return shuffle(r_v, random_state = 42)

#Retorna vetores para entrada de modelo
#  X valores e Y labels
#  Y label da imagem (1 = presença de doença)

def join(lateralidade):
  #print("Accessing files...")
  X=[]
  Y=[]
  for f in islice(openImgs(norm, lateralidade), 0, stop_n):
      X += [f]
      Y += [0]

  for f in islice(openImgs(doente, lateralidade), 0, stop_s):
      X += [f]
      Y += [1]
  return X, Y

def join_open_files(lateralidade=left, reshape=True):
  X, y = join(lateralidade)
  Xl = []
  for i in X:
    if reshape:
      Xl.append(cv.resize(cv.imread(i), (224, 224)))
    else:
      Xl.append(cv.imread(i))
  return np.array(Xl), np.array(y)

X, y = join_open_files() #Coloca as imagens já abertas nos vetores de 'treino e teste'

from keras.applications.vgg16 import VGG16
model = VGG16()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.5, random_state = 0)

score = 0
model.compile(optimizer='adam',loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
print(X_train.shape, y_train.shape)

model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(y_test)

import math
print(math.ceil(y_pred))
print(metrics.classification_report(y_test, y_pred))
#return X_train, X_test, y_train, y_test





(6, 224, 224, 3) (6,)


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


1/1 [==============================] - 1s 1s/step - loss: 13.8206 - accuracy: 0.0000e+00


[1 1 0 1 0 1]
[[1.0000000e+00 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 [1.0000000e+00 9.1440604e-37 0.0000000e+00 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 [1.0000000e+00 1.0552981e-34 0.0000000e+00 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 [1.0000000e+00 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 [1.0000000e+00 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 [1.0000000e+00 2.0365831e-37 0.0000000e+00 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]]
